In [ ]:
#please run on google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

#reading in data
df = pd.read_csv('drive/My Drive/adsTaxiTrips_2019_01.csv', dtype = str) #replace the path and file name
n = df.index[-1]
start_dates = [df["StartDateTime"][x] for x in range(n)]
origin_latitudes = [df["OriginLatitude"][x] for x in range(n)]
origin_longitudes = [df["OriginLongitude"][x] for x in range(n)]

In [ ]:
#method to parse the start dates
def date_to_num(s):
    a = s.split(" ")
    b = a[0].split("-")
    c = a[1].split(":")
    day = int(b[2])
    hour = int(c[0])
    minute = int(c[1])
    second = float(c[2])
    return 2920 + day * 96 + hour * 4 + (minute // 15)

In [ ]:
coords = []

for i in range(len(origin_latitudes)):
    coords.append([origin_latitudes[i], origin_longitudes[i]])

num_of_areas = 50

#clustering the data using the k-means clustering algorithm

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=num_of_areas)
kmeans.fit(coords)
clusters = kmeans.predict(coords)

clusters_to_coords_dict = {n: [] for n in range(num_of_areas)}
for x in range(len(clusters)):
    clusters_to_coords_dict[clusters[x]] = clusters_to_coords_dict[clusters[x]] + [coords[x]]

In [ ]:
clusters_to_mean = {n : "0" for n in range(num_of_areas)}

#finding the middle of each cluster

for i in range(num_of_areas):
    l = clusters_to_coords_dict[i]
    sum_lat = 0
    sum_long = 0
    for x in range(len(l)):
        sum_lat += float(l[x][0])
        sum_long += float(l[x][1])
    clusters_to_mean[i] = [sum_lat/len(l), sum_long/len(l)]

In [ ]:
time_sections = []

taxi_count = {}

#counting how many taxis are in each cluster during each time interval

for ind in range(len(start_dates)):
    n = date_to_num(start_dates[ind])
    time_sections.append(n)
    if n not in taxi_count.keys():
        taxi_count[n] = {clusters[x]:0 for x in range(50)}
        taxi_count[n][clusters[n]] = 1
    else:
        taxi_count[n][clusters[n]] += 1

In [ ]:
#writing out the dictionaries to a text file
with open('/content/drive/My Drive/January2019.txt', 'w') as writefile: #replace file name and path
    writefile.write("taxi_count = ")
    writefile.write(str(taxi_count))
    writefile.write("clusters_to_mean = ")
    writefile.write(str(clusters_to_mean))